In [0]:
%%shell

# pycocotoolsをインストール
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
python setup.py build_ext install

In [0]:
%%shell

# Penn-Fudan datasetをダウンロード
wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .
# 現在のフォルダに解凍
unzip PennFudanPed.zip

In [0]:
from PIL import Image
Image.open('PennFudanPed/PNGImages/FudanPed00001.png')

In [0]:
mask = Image.open('PennFudanPed/PedMasks/FudanPed00001_mask.png')
# 各マスクインスタンスには、0からNまでの異なる色がある。
# Nはインスタンスの数です。視覚化を簡単にするために、マスクにカラーパレットを追加する。
mask.putpalette([
    0, 0, 0, # black background
    255, 0, 0, # index 1 is red
    255, 255, 0, # index 2 is yellow
    255, 153, 0, # index 3 is orange
])
mask

In [0]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # すべての画像ファイルをロードし、並べ替えて整列させる
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # 各色は異なるインスタンスに対応し、0が背景であるため、マスクをRGBに変換していないことに注意
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # インスタンスは異なる色としてエンコードされる
        obj_ids = np.unique(mask)
        # 最初のIDは背景なので、削除
        obj_ids = obj_ids[1:]

        # カラーエンコードされたマスクをバイナリマスクのセットに分割
        masks = mask == obj_ids[:, None, None]

        # 各マスクの境界ボックス座標を取得
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # クラスは1つだけ
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # すべてのインスタンスが混雑していないとする
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [0]:
dataset = PennFudanDataset('PennFudanPed/')
dataset[0]

In [0]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
      
def get_instance_segmentation_model(num_classes):
    # COCOで事前トレーニングされたインスタンスセグメンテーションモデルをロードする
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # 分類器の入力特徴量の数を取得する
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # 事前に訓練されたヘッドを新しいヘッドと交換する
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # マスク分類器の入力特徴量の数を取得する
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # マスク予測器を新しいものに置き換える
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [0]:
%%shell

# TorchVisionリポジトリをダウンロード
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # 画像、PIL画像をPyTorchテンソルに変換
    transforms.append(T.ToTensor())
    if train:
        # トレーニング中、データ増強のためにトレーニング画像とグラウンドトゥルースをランダムに反転
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [0]:
# データセットとtransformを使用
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# 訓練とテストセットでデータセットを分割する
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# 訓練および検証データローダーを定義する
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# データセットには、背景と個人の2つのクラスのみがある
num_classes = 2

# ヘルパー関数を使用してモデルを取得
model = get_instance_segmentation_model(num_classes)
# モデルを適切なデバイスに移動
model.to(device)

# optimizerを構築
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# 3エポックごとに10倍ずつ学習率を下げる学習率スケジューラー
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [0]:
# 10 epochsで訓練
num_epochs = 10

for epoch in range(num_epochs):
    # 1エポックでトレーニングし、10回の反復ごとに表示
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # 学習率を更新
    lr_scheduler.step()
    # テストデータセットで評価
    evaluate(model, data_loader_test, device=device)

In [0]:
# テストセットから画像を1つ選択する
img, _ = dataset_test[0]
# モデルを評価モードにする
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

In [0]:
prediction

In [0]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [0]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())